# Tworzymy środowisko wirtualne i instalujemy wymagane pakiety

In [1]:
!python3 -m venv .env

### Pamiętaj o wybraniu interpretera ze środowiska .env (w przypadku VS Code)

In [2]:
!source .env/bin/activate

In [ ]:
!pip install --upgrade pip

!pip install gymnasium
!pip install 'gymnasium[box2d]'
!pip install torch
!pip install 'numpy<2'

### Ogólne przedstawienie problemu

In [5]:
import gymnasium as gym

env = gym.make("LunarLander-v2")

In [ ]:
env.reset()

terminated = False
truncated = False

while not (terminated or truncated):
    action = 1
    obs, reward, terminated, truncated, info = env.step(action)
    env.render()

env.close()

### Co zawiera stan zwracany przez środowisko?

- **pozycja pozioma** lądownika (x)
- **pozycja pionowa** lądownika (y)
- **prędkość pozioma** lądownika
- **prędkość pionowa** lądownika
- **kąt nachylenia** lądownika
- **prędkość kątowa** lądownika
- **czy noga nr 1 (lewa) lądownika ma styczność z podłożem**
- **czy noga nr 2 (prawa) lądownika ma styczność z podłożem**

### Ogólne dane dotyczące samego środowiska i możliwych akcji

In [9]:
print(f'Ilość możliwych akcji: {env.action_space.n}')

Ilość możliwych akcji: 4


### Jakie akcje są możliwe?
- **Akcja 0**: Brak działania
- **Akcja 1**: Uruchom główny silnik
- **Akcja 2**: Uruchom lewy boczny silnik
- **Akcja 3**: Uruchom prawy boczny silnik

### Ustalamy hardware pod obliczenia (CPU/GPU)

In [10]:
import torch

OPTIMIZE_WITH_HARDWARE = False

device = torch.device('cpu')
if OPTIMIZE_WITH_HARDWARE:
    if torch.backends.mps.is_available():
        device = torch.device('mps')
        print(f'Selected device: MPS (Metal Performance Shaders)')
    elif torch.backends.cuda.is_available():
        device = torch.device('cuda')
        print(f'Selected device: GPU with CUDA support')
else:
    print(f'Selected device: CPU')

Selected device: CPU


### Definiujemy strukturę naszej sieci głębokiej

In [1]:
import torch
from torch import nn

class DQNetwork(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, action_size)
    
    def forward(self, state):
        x = torch.nn.functional.leaky_relu(self.fc1(state), negative_slope=0.01)
        x = torch.nn.functional.leaky_relu(self.fc2(x), negative_slope=0.01)
        return self.fc3(x)

### Definiujemy obiekt agenta sieci DQN

In [2]:
from collections import deque
from torch import optim
import random
import numpy as np

# Hiperparametry treningu sieci DQN
LEARNING_RATE = 0.001
BATCH_SIZE = 16

class DQNAgent(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQNAgent, self).__init__()
        self.state_size = state_size        # ilość informacji dot. stanu środowiska
        self.action_size = action_size      # ilość akcji, które agent może wykonać
        self.discount_factor = 0.99         # współczynnik spadku wartości nagrody
        self.epsilon_greedy = 1.0           # początkowy współczynnik losowości (1 = 100% losowości)
        self.epsilon_greedy_min = 0.1       # minimalny współczynnik losowości
        self.epsilon_greedy_decay = 0.995   # zmniejszanie stopnia losowości co iterację o 5%
        self.memory = deque(maxlen=1000)    # kolekcja przechowująca 1000 ostatnich zdarzeń
        self.train_start = 500              # liczba zdarzeń, od której zaczynamy trenować model

        self.model = DQNetwork(state_size, action_size)
        self.optimizer = optim.Adam(self.model.parameters(), lr=LEARNING_RATE)
        self.criterion = nn.MSELoss()

    # Zapisuje podjętą akcję w danym stanie i jej skutki 
    def memorize(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    # Wybiera akcje dla danego stanu. Jeśli aktualnie model
    # nie eksploruje (wykonuje losową akcje) to wybierana jest
    # akcja o najlepszym potencjale (najwyższa wartość nagrody)
    def act(self, state):
        if np.random.rand() <= self.epsilon_greedy:
            return random.randrange(self.action_size)
        # unsqueeze zapewnia odpowiedni wymiar [batch_size, state_size]
        # PyTorch narzuca format danych treningowych w postaci tensora, który
        # w pierwszym wymiarze zawiera informację i ilości paczek a następnie same
        # dane treningowe, dlatego 'unsqueeze' rozszerza wymiar danych mimo tego, że
        # mamy tylko jedną paczkę w tej funkcji
        state = torch.FloatTensor(state).unsqueeze(0)
        with torch.no_grad():
            q_values_predicted = self.model(state)
        return torch.argmax(q_values_predicted).item()
    

    def replay(self):
        # Nie zaczynamy trenować modelu dopóki nie zbierzemy
        # minimalnej ilości danych w buforze memory
        if len(self.memory) < self.train_start:
            return
        
        data_batch = random.sample(self.memory, BATCH_SIZE) # Losujemy paczkę danych do treningu
        
        total_mse_loss = 0
        for state, action, reward, next_state, done in data_batch:
            state = torch.FloatTensor(state)
            next_state = torch.FloatTensor(next_state)
            reward = torch.FloatTensor([reward])
            discounted_reward = reward
            if not done:
                discounted_reward += self.discount_factor * torch.max(self.model(next_state))
            
            dqn_prediction = self.model(state)
            true_reward = dqn_prediction.clone()     # Tworzymy klon aby nadpisać wynik dla akcji niżej
            true_reward[action] = discounted_reward  # Nadpisujemy wartość nagrody dla wykonanej akcji
            
            loss = self.criterion(dqn_prediction, true_reward)
            
            self.optimizer.zero_grad()  # Zerujemy gradient
            loss.backward()             # Liczymy gradient
            self.optimizer.step()       # Aktualizujemy wagi sieci

            total_mse_loss += loss.item()
        
        # Jeśli nie doszliśmy do minimalnej wartości współczynnika
        # eksploracji to nadal go zmniejszamy z każdą iteracją
        if self.epsilon_greedy > self.epsilon_greedy_min:
            self.epsilon_greedy *= self.epsilon_greedy_decay
        
        return total_mse_loss / BATCH_SIZE # zwracamy średni błąd MSE

### Funkcja do zapisu oraz odtworzenia naszego modelu, aby nie stracić danych podczas treningu

In [3]:
def save_checkpoint(model, optimizer, episode, filename="model_checkpoint.pth"):
    checkpoint = {
        'episode': episode,  # Numer epoki, aby można było wznowić trening
        'model_state_dict': model.state_dict(),  # Wagi modelu
        'optimizer_state_dict': optimizer.state_dict()  # Parametry optymalizatora
    }
    torch.save(checkpoint, filename)  # Zapisujemy do pliku

def load_checkpoint(model, optimizer, filename):
    checkpoint = torch.load(filename)
    model.load_state_dict(checkpoint['model_state_dict'])  # Załaduj wagi modelu
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])  # Załaduj stan optymalizatora
    episode = checkpoint['episode']  # Załaduj numer epoki
    return episode

### Definiujemy pętlę treningową modelu DQN

In [ ]:
import gymnasium as gym
import math

env = gym.make("LunarLander-v2", render_mode="human")
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)

# Kolekcje do śledzenia postępów treningu
rewards_history = []
epsilon_history = []
loss_history = []

episodes = 1000
for episode in range(episodes):
    state, _ = env.reset()
    done = False
    
    total_reward = 0
    total_mse_loss = 0
    step_counter = 0
    
    while not done:
        action = agent.act(state)
        next_state, reward, done, _, _ = env.step(action)
        total_reward += reward
        
        agent.memorize(state, action, reward, next_state, done)
        state = next_state
        step_counter += 1

        mse_loss = agent.replay()
        if mse_loss is not None:
            total_mse_loss += mse_loss
        
        if done:
            average_loss = total_mse_loss / step_counter if step_counter > 0 else 0
            print(f"Episode: {episode+1}/{episodes}, Reward: {total_reward:.2f}, 
                  Epsilon: {agent.epsilon_greedy:.3f}, MSE: {total_mse_loss:.3f} 
                  Loss: {math.sqrt(total_mse_loss):.2f}")

            # Zapisujemy dane opisujące postępy treningu
            rewards_history.append(total_reward)
            epsilon_history.append(agent.epsilon_greedy)
            loss_history.append(math.sqrt(average_loss))
            break
    
    # Zapisujemy nasz trenowany model co 10 epizod
    if episode % 10 == 0:
        save_checkpoint(agent.model, agent.optimizer, 
                        episode, filename=f"model_checkpoint_{episode}.pth")

env.close()